## Import Libraries

In [2]:
%matplotlib inline
from utils import *

from keras.layers import Activation,TimeDistributed
from numpy.random import choice

## Setup

In [3]:
path = '../../data/nietzsche/nietzsche.txt';
text = open(path).read().lower()
print('Corpus Length : ',len(text))

Corpus Length :  600893


In [7]:
!tail -n25 $path 

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [9]:
chars = sorted(list(set(text)));
vocab_size=len(chars)+1
print('Total unique chars : ',vocab_size)

Total unique chars :  58


In [11]:
chars.insert(0,'\0')

In [13]:
''.join(chars[:])

'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [15]:
char_indices={v:k for k,v in enumerate(chars)}
indices_char={k:v for k,v in enumerate(chars)}

In [19]:
idx=[char_indices[c] for c in text]

In [20]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [22]:
''.join([indices_char[a] for a in idx[:70]])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [128]:
maxlen=40
sentences = [];
next_chars = [];
for i in range(0,len(text)-maxlen+1):
    tmp1=text[i:i+maxlen];
    aa=[abc for abc in tmp1];
    sentences.append(aa);
    tmp2=text[i+1:maxlen+1];
    aa=[abc for abc in tmp1];   
    next_chars.append(aa);

In [129]:
print('nb sequences : ',len(sentences))

nb sequences :  600854


In [130]:
np.shape(sentences)

(600854, 40)

In [139]:
n_fac=24

In [140]:
model = Sequential([
    Embedding(vocab_size,n_fac,input_length=maxlen),
    LSTM(512,input_shape=(None,n_fac),return_sequences=True,dropout=0.2,recurrent_dropout=0.2,
        implementation=2),
    Dropout(0.2),
    LSTM(512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,
        implementation=2),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')
])

In [141]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 24)            1392      
_________________________________________________________________
lstm_9 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 40, 512)           2099200   
_________________________________________________________________
dropout_10 (Dropout)         (None, 40, 512)           0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 40, 58)            29754     
_________________________________________________________________
activation_5 (Activation)    (None, 40, 58)            0         
Total para

In [142]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [149]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(1):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [146]:
x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]

In [145]:
seed_string="ethics is a basic foundation of all that"

In [153]:
model.fit(sentences,np.expand_dims(next_chars,2),batch_size=64,epochs=1)

Epoch 1/1


ValueError: could not convert string to float: 'n'